## Lecture 10:


* We are going to use **_CNN_** in this Lecture
* The dataset to be used is the **_MINST Dataset_** given by pytorch package

In [2]:
# Importing the stock libraries

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [3]:
# Creating the CUDA environment

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
# Creating the dataset for the model

train_dataset = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=True)

In [5]:
# Creating the dataloader for the input to the model

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [6]:
# Creating the Model

class Model(torch.nn.Module):

    ## Initiatize
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
        self.l1 = torch.nn.Linear(320, 10)

        self.max = torch.nn.MaxPool2d(2)


    ## Forward
    def forward(self, x):
        in_size = x.size(0)
        x = torch.nn.functional.relu(self.max(self.conv1(x)))
        x = torch.nn.functional.relu(self.max(self.conv2(x)))
        x = x.view(in_size, -1)
        x = self.l1(x)
        return torch.nn.functional.log_softmax(x)

model = Model()
model.to(device)

Model(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (l1): Linear(in_features=320, out_features=10, bias=True)
  (max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [7]:
# Define the loss function and optimizer

criterion = torch.nn.CrossEntropyLoss(reduction='mean')
optimus = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.05)

In [8]:
# This is the code for training

def train(epoch):
    model.train()
    for _, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        predict = model(inputs)

        # loss
        loss = criterion(predict, labels)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        # zero optimus,back propagation, update optimus
        optimus.zero_grad()
        loss.backward()
        optimus.step()


In [9]:
# This code is for the validation step

def valid(test_dataloader):
    model.eval()
    total=0; n_correct =0 
    with torch.no_grad():
        for _, data in enumerate(test_dataloader,0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            if _%500 == 0:
                print(f'Validation completed for {_} samples')

            predict = model(inputs)
            total += labels.size(0)
            big_val, big_idx = torch.max(predict.data, dim=1)
            n_correct += (big_idx==labels).sum().item()

    return (n_correct*100)/total

In [10]:
# Calling the training function

print('Starting the training')

for epoch in range(0,2):
    train(epoch)

print('Training Completed')

print('+++++++++++++++++=================+++++++++++++++++=================++++++++++++++++')

print('Starting the testing')
accu = valid(test_dataloader)
print(f'The accuracy of the model is: {accu}')

Starting the training
Epoch: 0, Loss:  2.313748598098755
Epoch: 0, Loss:  0.3788803517818451
Epoch: 0, Loss:  0.35306981205940247
Epoch: 0, Loss:  0.15131986141204834
Epoch: 1, Loss:  0.48513269424438477
Epoch: 1, Loss:  0.13334602117538452
Epoch: 1, Loss:  0.1520206332206726
Epoch: 1, Loss:  0.08754238486289978
Training Completed
+++++++++++++++++=================+++++++++++++++++=================++++++++++++++++
Starting the testing
Validation completed for 0 samples
The accuracy of the model is: 96.57


C:\Users\abhis\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
